In [1]:
cd ..

/media/sanjyot/DATA/UMich google drive/Academics/Fall 2020/Data Security and Privacy - CIS 545/project/EvaluatingDPML


In [2]:
%load_ext autoreload
%autoreload 2

In [14]:
import numpy as np

from attack import load_data, train_target_model, yeom_membership_inference
from utilities import log_loss
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
DATA_PATH = 'data/'
data_name = 'target_data.npz'

In [24]:
class Args:
    train_dataset = None
    run = 1
    use_cpu = 0
    save_model = 0
    save_data = 0
    n_shadow = 5
    target_data_size = 1e4
    target_test_train_ratio = 1
    target_model = 'nn'
    target_learning_rate = 0.01
    target_batch_size = 200
    target_n_hidden = 256  # 256
    target_epochs = 50
    target_l2_ratio = 1e-8
    target_clipping_threshold = 1
    target_privacy = 'no_privacy'
    target_dp = 'dp'
    target_epsilon = 0.5
    target_delta = 1e-5
    attack_model = 'softmax'
    attack_learning_rate = 0.01
    attack_batch_size = 100
    attack_n_hidden = 64
    attack_epochs = 100
    attack_l2_ratio = 1e-6
    
args = Args()

In [25]:
dataset = load_data(data_name, args)
train_x, train_y, test_x, test_y = dataset

In [26]:
true_x = np.vstack((train_x, test_x))
true_y = np.append(train_y, test_y)

## Non private

In [21]:
pred_y, membership, test_classes, classifier, aux = train_target_model(
        args=args,
        dataset=dataset,
        epochs=args.target_epochs,
        batch_size=args.target_batch_size,
        learning_rate=args.target_learning_rate,
        clipping_threshold=args.target_clipping_threshold,
        n_hidden=args.target_n_hidden,
        l2_ratio=args.target_l2_ratio,
        model=args.target_model,
        privacy=args.target_privacy,
        dp=args.target_dp,
        epsilon=args.target_epsilon,
        delta=args.target_delta,
        save=args.save_model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpcbsmnzpo', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.


In [22]:
train_loss, train_acc, test_loss, test_acc = aux

In [10]:
print(train_acc, test_acc)

0.9999 0.1679


In [15]:
per_instance_loss = np.array(log_loss(true_y, pred_y))

In [17]:
pred_membership = yeom_membership_inference(per_instance_loss, membership, train_loss)

----------YEOM'S MEMBERSHIP INFERENCE----------

TP: 7305     FP: 529     FN: 2695     TN: 9471
PPV: 0.9325
Advantage: 0.6776


PPV = Precision

Advantage = TPR - FPR